# Introduction

Baseline experiment

# Set up Environment

In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.feature_selection import SelectFromModel
from math import sqrt
from sklearn.metrics import (mean_squared_error, mean_absolute_error, r2_score,
                             explained_variance_score, max_error)

# display all columns of dataframes in the notebook
pd.options.display.max_columns = None

# ignore warnings
warnings.filterwarnings('ignore')

# set up random seed for reproducibility
RANDOM_SEED = 42

# Load Data

In [38]:
file_path = '../data/interim/'
file_name = 'train_val_data.csv'
flights = pd.read_csv(file_path + file_name)

# Feature Engineering

In [39]:
flights.head(3)

,AOBTtoAIBT,UniqueCarrierCode,OriginAirportID,OriginCityMarketID,OriginState,OBTDelay,OBTDel15,OBTDelayGroups,SOBTtoSIBT,Distance,DistanceGroup,Num_Arr_SIBT-30,Num_Arr_SIBT-25,Num_Arr_SIBT-20,Num_Arr_SIBT-15,Num_Arr_SIBT-10,Num_Arr_SIBT-5,Num_Arr_SIBT-0,Num_Arr_SIBT+5,Num_Arr_SIBT+10,Num_Arr_SIBT+15,Num_Arr_SIBT+20,Num_Arr_SIBT+25,Num_Dep_SIBT-30,Num_Dep_SIBT-25,Num_Dep_SIBT-20,Num_Dep_SIBT-15,Num_Dep_SIBT-10,Num_Dep_SIBT-5,Num_Dep_SIBT-0,Num_Dep_SIBT+5,Num_Dep_SIBT+10,Num_Dep_SIBT+15,Num_Dep_SIBT+20,Num_Dep_SIBT+25,SIBTQuarter,SIBTMonth,SIBTDayOfMonth,SIBTDayOfWeek,SIBTHour
0,218,DL,14869,34614,UT,-2,0,-1,223,1590,7,0,0,0,0,1,0,0,2,0,0,0,0,2,1,1,0,1,0,0,3,1,0,3,1,1,1,1,7,6
1,81,F9,13204,31454,FL,-4,0,-1,88,404,2,0,0,0,0,3,0,1,0,0,0,0,1,1,1,0,1,3,0,3,1,0,3,1,0,1,1,1,7,6
2,88,EV,10980,30980,TN,302,1,12,57,106,1,0,0,0,0,3,0,0,0,0,0,1,0,1,0,1,0,4,0,2,0,3,1,0,0,1,1,1,7,6


## Separate Train and Validation Datasets

In [ ]:
# the last 10000 flights are set as validation dataset
X_train, X_val, y_train, y_val = train_test_split(flights,
                                                  flights['AOBTtoAIBT'],
                                                  shuffle=False,
                                                  test_size=10000)

X_train.shape, X_val.shape

In [ ]:
X_val.head(3)

The validation set are flights after 2018-12-21 6pm.

## Temporal Variables

In [ ]:
temporal_var = ['SIBTQuarter', 'SIBTMonth',
                'SIBTDayOfMonth', 'SIBTDayOfWeek', 'SIBTHour']


def analyse_temporal_var(df, var):

    # function plots median actual flight duration by every temporal var

    temp = df.copy()
    temp.groupby(var)['AOBTtoAIBT'].median().plot.bar()
    plt.title(var)
    plt.ylabel('AOBTtoAIBT')
    plt.show()


for var in temporal_var:
    analyse_temporal_var(X_train, var)

## Categorical Variables

In [ ]:
# capture categorical variables in a list
cat_vars = [var for var in X_train.columns if X_train[var].dtypes == 'O']

print('Number of categorical variables: ', len(cat_vars))
print(cat_vars)

### Cardinality

In [ ]:
# cardinality
X_train[cat_vars].nunique()

UniqueCarrierCode shows low cardinality.

### Rare Labels

In [ ]:
def analyse_rare_labels(df, var, rare_perc):

    # determine the % of observations per category
    tmp = df.groupby(var)['UniqueCarrierCode'].count() / len(df)

    # return categories that are rare
    return tmp[tmp < rare_perc]


# print categories that are present in less than
# 1 % of the observations
rare_perc = 0.01
for var in cat_vars:
    print(analyse_rare_labels(X_train, var, rare_perc))

These airlines are present in less than 1% of flights in ATL in 2017 and 2018. Labels that are under-represented in the dataset tend to cause over-fitting of models. They will be removed.

In [ ]:
def find_frequent_labels(df, var, rare_perc):
    # function finds the labels that are shared by more than
    # a certain % of the houses in the dataset
    tmp = df.groupby(var)['UniqueCarrierCode'].count() / len(df)
    return tmp[tmp >= rare_perc].index


for var in cat_vars:

    # find the frequent categories
    frequent_ls = find_frequent_labels(X_train, var, rare_perc)

    # replace rare categories by the string "Rare"
    X_train[var] = np.where(X_train[var].isin(
        frequent_ls), X_train[var], 'Rare')
    X_val[var] = np.where(X_val[var].isin(
        frequent_ls), X_val[var], 'Rare')

### Encode

Next, transform the strings of the categorical variables into numbers, so that we capture the monotonic relationship between the label and the target.

In [ ]:
# this function will assign discrete values to the strings of the variables,
# so that the smaller value corresponds to the category that shows the smaller
# mean AOBTtoAIBT


def replace_categories(train, val, var, target):

    # order the categories in a variable from that with the lowest
    # house sale price, to that with the highest
    ordered_labels = train.groupby([var])[target].mean().sort_values().index

    # create a dictionary of ordered categories to integer values
    ordinal_label = {k: i for i, k in enumerate(ordered_labels, 0)}

    # use the dictionary to replace the categorical strings by integers
    train[var] = train[var].map(ordinal_label)
    val[var] = val[var].map(ordinal_label)


for var in cat_vars:
    replace_categories(X_train, X_val, var, 'AOBTtoAIBT')

## Feature Scaling

For use in linear models, features need to be either scaled or normalised. In the next section, I will scale features to the minimum and maximum values:

In [ ]:
X_train = X_train[X_train.columns[1:37]]
X_val = X_val[X_val.columns[1:37]]

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

# Feature Selection

In [ ]:
# remember to set the seed, the random state in this function
sel_ = SelectFromModel(Lasso(alpha=0.005, random_state=RANDOM_SEED))

# train Lasso model and select features
sel_.fit(X_train, y_train)

In [ ]:
# let's print the number of total and selected features

# this is how we can make a list of the selected features
selected_feats = flights.iloc[:, 1:37].columns[(sel_.get_support())]

# let's print some stats
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feats)))
print('features with coefficients shrank to zero: {}'.format(
    np.sum(sel_.estimator_.coef_ == 0)))

In [ ]:
# print the selected features
selected_feats

In [ ]:
feats = selected_feats.to_list()

In [ ]:
X_train = np.delete(X_train, [17, 21, 29], 1)
X_val = np.delete(X_val, [17, 21, 29], 1)

# Model

## Lasso Regression

In [ ]:
# set up the model
lasso = Lasso(alpha=0.005, random_state=RANDOM_SEED)

# train the model
lasso.fit(X_train, y_train)

### Evalutation of Lasso

In [ ]:
# evaluate performance using the RMSE, MAE and R2

# make predictions for train set
pred_train = lasso.predict(X_train)

# determine metrics
print('train rmse: {:.3f}'.format(sqrt(mean_squared_error(y_train, pred_train))))
print('train mae: {:.3f}'.format(mean_absolute_error(y_train, pred_train)))
print('train r2: {:.3f}'.format(r2_score(y_train, pred_train)))
print('train explained variance score: {:.3f}'.format(
    explained_variance_score(y_train, pred_train)))
print('val max error: {:.3f}'.format(max_error(y_train, pred_train)))
print()

# make predictions for validation set
pred_val = lasso.predict(X_val)

# determine metrics
print('val rmse: {:.3f}'.format(sqrt(mean_squared_error(y_val, pred_val))))
print('val mae: {:.3f}'.format(mean_absolute_error(y_val, pred_val)))
print('val r2: {:.3f}'.format(r2_score(y_val, pred_val)))
print('val explained variance score: {:.3f}'.format(
    explained_variance_score(y_val, pred_val)))
print('val max error: {:.3f}'.format(max_error(y_val, pred_val)))
print()

sch_val = flights['SOBTtoSIBT'][-10000:]
# determine metrics
print('sch rmse: {:.3f}'.format(sqrt(mean_squared_error(y_val, sch_val))))
print('sch mae: {:.3f}'.format(mean_absolute_error(y_val, sch_val)))
print('sch r2: {:.3f}'.format(r2_score(y_val, sch_val)))
print('sch explained variance score: {:.3f}'.format(
    explained_variance_score(y_val, sch_val)))
print('val max error: {:.3f}'.format(max_error(y_val, sch_val)))

In [ ]:
# evaluate our predictions respect to actual durations
plt.figure(figsize=(5,5))
plt.scatter(y_val, pred_val)
plt.xlabel('AOBTtoAIBT')
plt.ylabel('Predicted AOBTtoAIBT')
plt.title('Evaluation of Lasso Predictions')

In [ ]:
# let's evaluate the distribution of the errors:
# they should be fairly normally distributed
errors = y_val - lasso.predict(X_val)
errors.hist(bins=30)

The distribution of the errors follows a gaussian distribution. That suggests that our model is doing something useful.

In [ ]:
y_val_series = y_val.reset_index(drop=True)
# sch_val_series = sch_val.reset_index(drop=True)
pred_val_series = pd.Series(pred_val)
durations = pd.concat([y_val_series, pred_val_series], axis=1)
# durations = pd.concat([durations, sch_val_series], axis=1)

# durations = durations.rename(columns={'AOBTtoAIBT': 'Actual', 0: 'Predicted', 'SOBTtoSIBT': 'Scheduled'})
durations = durations.rename(columns={'AOBTtoAIBT': 'Actual', 0: 'Predicted'})

first_50 = durations.head(50)
first_50.plot(kind='bar',figsize=(14,8))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

### Feature Importance

In [ ]:
# Finally, look at the feature importance
importance = pd.Series(np.abs(lasso.coef_.ravel()))
importance.index = feats
importance.sort_values(inplace=True, ascending=False)
importance.plot.bar(figsize=(18, 6))
plt.ylabel('Lasso Coefficients')
plt.title('Feature Importance')

## Ridge Regression

### Evaluation of Ridge

In [ ]:
# set up the model
ridge = Ridge(alpha=0.005, random_state=RANDOM_SEED)

# train the model
ridge.fit(X_train, y_train)

In [ ]:
# evaluate performance using the RMSE, MAE and R2

# make predictions for train set
pred_train = ridge.predict(X_train)

# determine metrics
print('train rmse: {:.3f}'.format(sqrt(mean_squared_error(y_train, pred_train))))
print('train mae: {:.3f}'.format(mean_absolute_error(y_train, pred_train)))
print('train r2: {:.3f}'.format(r2_score(y_train, pred_train)))
print('train explained variance score: {:.3f}'.format(
    explained_variance_score(y_train, pred_train)))
print('val max error: {:.3f}'.format(max_error(y_train, pred_train)))
print()

# make predictions for validation set
pred_val = ridge.predict(X_val)

# determine metrics
print('val rmse: {:.3f}'.format(sqrt(mean_squared_error(y_val, pred_val))))
print('val mae: {:.3f}'.format(mean_absolute_error(y_val, pred_val)))
print('val r2: {:.3f}'.format(r2_score(y_val, pred_val)))
print('val explained variance score: {:.3f}'.format(
    explained_variance_score(y_val, pred_val)))
print('val max error: {:.3f}'.format(max_error(y_val, pred_val)))
print()

sch_val = flights['SOBTtoSIBT'][-10000:]
# determine metrics
print('sch rmse: {:.3f}'.format(sqrt(mean_squared_error(y_val, sch_val))))
print('sch mae: {:.3f}'.format(mean_absolute_error(y_val, sch_val)))
print('sch r2: {:.3f}'.format(r2_score(y_val, sch_val)))
print('sch explained variance score: {:.3f}'.format(
    explained_variance_score(y_val, sch_val)))
print('val max error: {:.3f}'.format(max_error(y_val, sch_val)))

### Feature Importance

In [ ]:
# Finally, look at the feature importance
importance = pd.Series(np.abs(ridge.coef_.ravel()))
importance.index = feats
importance.sort_values(inplace=True, ascending=False)
importance.plot.bar(figsize=(18, 6))
plt.ylabel('Ridge Coefficients')
plt.title('Feature Importance')